In [14]:
import contextlib
import os
import time
from IPython.display import Audio
import pyaudio
import wave
from windows_volume_controller import MasterVolumeController
import pickle
from pydub import AudioSegment
from pydub.playback import play

VOLUME_INFO_FILENAME = "volume_info.dat"
INITIAL_RECORDING_FILENAME = "initial.wav"
PLAYBACK_RECORDING_FILENAME = "playback.wav"
CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 2
SAMPLE_RATE = 48000
RECORD_SECONDS = 10

volume_controller = MasterVolumeController()

with contextlib.suppress(OSError):
    os.remove(VOLUME_INFO_FILENAME)

In [18]:
def record_audio(filename):
    # initialize PyAudio object
    p = pyaudio.PyAudio()
    # open stream object as input & output
    stream = p.open(format=FORMAT,
                    channels=CHANNELS,
                    rate=SAMPLE_RATE,
                    input=True,
                    output=True,
                    frames_per_buffer=CHUNK)
    frames = []
    print("Getting current master volume in dB...")
    current_vol = volume_controller.get_volume_db()
    print("Saving volume info...")
    with open(VOLUME_INFO_FILENAME, 'wb') as f:
        pickle.dump(current_vol, f)
    print(f"Recording for {RECORD_SECONDS} seconds...")
    for _ in range(int(SAMPLE_RATE / CHUNK * RECORD_SECONDS)):
        data = stream.read(CHUNK)
        # if you want to hear your voice while recording
        # stream.write(data)
        frames.append(data)
    print("Finished recording.")
    # stop and close stream
    stream.stop_stream()
    stream.close()
    # terminate pyaudio object
    p.terminate()
    # save audio file
    # open the file in 'write bytes' mode
    print(f"Saving to audio file '{filename}'...")
    wf = wave.open(filename, "wb")
    # set the channels
    wf.setnchannels(CHANNELS)
    # set the sample format
    wf.setsampwidth(p.get_sample_size(FORMAT))
    # set the sample rate
    wf.setframerate(SAMPLE_RATE)
    # write the frames as bytes
    wf.writeframes(b"".join(frames))
    print("Done")
    # close the file
    wf.close()

def play_audio(filename):
    playback_audio = AudioSegment.from_file(filename)
    print("Getting initially recorded volume...")
    with open(VOLUME_INFO_FILENAME, 'rb') as f:
        recorded_vol = pickle.load(f)
    print(f"Playing on current volume of {volume_controller.get_volume_db()} dB")
    play(playback_audio)
    print("Done")
    time.sleep(2)
    print(f"Resetting to initial volume of {recorded_vol} dB")
    volume_controller.set_volume_db(recorded_vol)
    print("Playing...")
    play(playback_audio)
    print("Done")


In [19]:
# Record & Save Audio and Volume
record_audio(INITIAL_RECORDING_FILENAME)
Audio(INITIAL_RECORDING_FILENAME)

Getting current master volume in dB...
Saving volume info...
Recording for 10 seconds...
Finished recording.
Saving to audio file 'initial.wav'...
Done


In [21]:
# Normalized Playback
play_audio(INITIAL_RECORDING_FILENAME)

Getting initially recorded volume...
Playing on current volume of -37.88519287109375 dB
Done
Resetting to initial volume of -10.329694747924805 dB
Playing...
Done
